# Emotion detection outputs
In this notebook, a small example will be shown to show how the scoring system of this retrieval based system will operate.
For this, the self made database will be used with some small example sentences.

In [1]:
# install necessary libraries
import sqlite3 as sql
from google.colab import drive
import random as rd

The first step is connect to the drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path = "drive/MyDrive/HBO/cursussen/jaar_4/afstuderen/ed_outputs.db"

In [4]:
# make connection to the database
conn = sql.connect(path)
cur = conn.cursor()

In [5]:
# database content
cur.execute("SELECT id, output, emotion, score FROM outputs")
cur.fetchall()

[(1, 'Laten we de situatie bespreken en een oplossing vinden.', 'angry', 0),
 (2,
  ' Ik begrijp dat je het niet leuk vindt. Ik ge voor jou een oplossing zoeken.',
  'angry',
  0),
 (3,
  'Laten we kijken wat er aan de hand is en bekijken hoe we het kunnen oplossen.',
  'angry',
  0),
 (4,
  'Laten we kalm blijven en samen naar een bruikbare oplossing zoeken.',
  'angry',
  0),
 (5,
  'Als er zaken zijn die moeten worden opgelost, laten we ze uitzoeken  en aanpakken.',
  'angry',
  0),
 (6, 'Fijn om te horen dat je tevreden bent!', 'joy', 0),
 (7, 'Het is goed om te horen dat je zo blij bent!', 'joy', 0),
 (8, 'Dat is goed nieuws, fijn!', 'joy', 0),
 (9, 'Graag gedaan, blij dat ik kan helpen!', 'joy', 0),
 (10, 'Wat leuk!', 'joy', 0),
 (11, 'Laat me weten hoe ik kan helpen.', 'sad', 0),
 (12, 'Als je iets nodig hebt, aarzel dan niet om te vragen.', 'sad', 0),
 (13, 'We gaan samen een oplossing zoeken..', 'sad', 0),
 (14, 'Als je hulp nodig hebt, laat het me weten.', 'sad', 0),
 (15, 'A

As grading method, the markov decision process will be used. This is a easy, yet efficient method to make the retrieval based system choose a output. The score will be graded using the reply **after** sending the output. the grading works as follow:

1. negative --> negative = -1
2. negative --> neutral = +1
3. negative --> positive = +2
4. neutral -->  neutral = 0
5. neutral --> positive = +1
6. neutral --> negative = -1
7. positive --> positve = +1
8. positive --> neutral = 0
9. positive --> negative = -2

In [6]:
# 4 sentences to show how a retrieval based works".
conv_user = [{"text": "Hallo, hoe gaat het", "emotion": "neutral"},
             {"text": "Domme chatbot", "emotion": "angry"},
             {"text": "Dat zou fijn zijn", "emotion": "neutral"},
             {"text": "ja, Heel erg bedankt", "emotion": "joy"}
             ]


In [53]:
class RetrievalOutput:
  def __init__(self):
    self.database = conn,
    self.cur = cur
    self.memory = []
    self.conv_id = 0
    self.epsilon = 0.9
    self. categories = {"pos": ["joy", "love"], "neu": ["neutral", "other"], "neg": ["sad", "angry", "fear"]}
    self.grading = {"neg-neg": -1, "neg-neu": 1, "neg-pos": 2, "neu-neu": 0, "neu-pos": 1, "neu-neg": -1, "pos-pos": 1, "pos-neu": 0, "pos-neg": -2}

  def check_emotion(self, u_input):
    #in this function the input would be fowarded to the ed model to determine a emotion, but for this example the emotion is pre determined
    emotion = u_input['emotion']
    return emotion

  def get_output(self, emotion):
    # retrieve from db using policy greedy
    if emotion != "neutral":
      rd_int = round(rd.random(), 2)
      print(rd_int)
      if rd_int > self.epsilon:
        # random
        query = "SELECT id, output, emotion, score FROM outputs WHERE emotion = '%s' ORDER BY RANDOM() LIMIT 1" % emotion
      else:
        # not random
        query = "SELECT id, output, emotion, score FROM outputs WHERE emotion = '%s' ORDER BY score DESC LIMIT 1" % emotion

      output = self.cur.execute(query)
      output = list(self.cur.fetchone())

    else:
      output = "test"

    return output

  def save(self, input, rb_output, emotion):
    # save conversation as dict for grading

    # get correct category using emotion
    for key, value in self.categories.items():
      for x in value:
        if x == emotion:
          category = key

    conversation = {"conv_id": self.conv_id, "input": input, "rb": rb_output, "pnn": category}
    self.memory.append(conversation)

  def grade(self):
    # grade previous conversation using current conversation
     sentiment = self.memory[self.conv_id]['pnn']
     if len(self.memory) > 1:
      prev_conv = self.memory[self.conv_id - 1]
      prev_sentiment = prev_conv['pnn']

      change = prev_sentiment + "-" + sentiment
      grade = self.grading[change]
      print(change, grade)
      if prev_sentiment != "neu":
        prev_conv['rb'][3] = prev_conv['rb'][3] + grade
        self.update_db(prev_conv['rb'])

     else:
      print('first conv')


     self.conv_id += 1

  def update_db(self, response):
    query = "UPDATE outputs SET score = %s WHERE id = %s" % (response[3], response[0])
    cur.execute(query)

  def reset_score(self):
    query = "UPDATE outputs SET score = 0"
    cur.execute(query)

  def reset_memory(self):
    self.memory.clear()

  def forward(self, text):
    # return response.
    emotion =  self.check_emotion(text)
    rb_output =  self.get_output(emotion)
    self.save(text, rb_output, emotion)
    self.grade()

    if rb_output == "test":
      return rb_output

    else:
      return rb_output[1]

The code below gives a example of a small conversation, by rerunning the two cells, you can see the score increasing. using the epsilon variable in the class function, u can make it more random for this testing process to see different outputs be graded more often.

In [55]:
res =  RetrievalOutput()
for conv in conv_user:
  print("input",conv)
  print("res",res.forward(conv))

input {'text': 'Hallo, hoe gaat het', 'emotion': 'neutral'}
first conv
res test
input {'text': 'Domme chatbot', 'emotion': 'angry'}
0.94
neu-neg -1
res Laten we de situatie bespreken en een oplossing vinden.
input {'text': 'Dat zou fijn zijn', 'emotion': 'neutral'}
neg-neu 1
res test
input {'text': 'ja, Heel erg bedankt', 'emotion': 'joy'}
0.72
neu-pos 1
res Fijn om te horen dat je tevreden bent!
[{'conv_id': 0, 'input': {'text': 'Hallo, hoe gaat het', 'emotion': 'neutral'}, 'rb': 'test', 'pnn': 'neu'}, {'conv_id': 1, 'input': {'text': 'Domme chatbot', 'emotion': 'angry'}, 'rb': [1, 'Laten we de situatie bespreken en een oplossing vinden.', 'angry', 3], 'pnn': 'neg'}, {'conv_id': 2, 'input': {'text': 'Dat zou fijn zijn', 'emotion': 'neutral'}, 'rb': 'test', 'pnn': 'neu'}, {'conv_id': 3, 'input': {'text': 'ja, Heel erg bedankt', 'emotion': 'joy'}, 'rb': [6, 'Fijn om te horen dat je tevreden bent!', 'joy', 0], 'pnn': 'pos'}]
[]


In [56]:
cur.execute("SELECT id, output, emotion, score FROM outputs")
cur.fetchall()

[(1, 'Laten we de situatie bespreken en een oplossing vinden.', 'angry', 3),
 (2,
  ' Ik begrijp dat je het niet leuk vindt. Ik ge voor jou een oplossing zoeken.',
  'angry',
  0),
 (3,
  'Laten we kijken wat er aan de hand is en bekijken hoe we het kunnen oplossen.',
  'angry',
  0),
 (4,
  'Laten we kalm blijven en samen naar een bruikbare oplossing zoeken.',
  'angry',
  0),
 (5,
  'Als er zaken zijn die moeten worden opgelost, laten we ze uitzoeken  en aanpakken.',
  'angry',
  0),
 (6, 'Fijn om te horen dat je tevreden bent!', 'joy', 0),
 (7, 'Het is goed om te horen dat je zo blij bent!', 'joy', 0),
 (8, 'Dat is goed nieuws, fijn!', 'joy', 0),
 (9, 'Graag gedaan, blij dat ik kan helpen!', 'joy', 0),
 (10, 'Wat leuk!', 'joy', 0),
 (11, 'Laat me weten hoe ik kan helpen.', 'sad', 0),
 (12, 'Als je iets nodig hebt, aarzel dan niet om te vragen.', 'sad', 0),
 (13, 'We gaan samen een oplossing zoeken..', 'sad', 0),
 (14, 'Als je hulp nodig hebt, laat het me weten.', 'sad', 0),
 (15, 'A